# Dividends vs Growth

Comparing the approaches of investing for dividends or growth over time

In [96]:
import pandas as pd
import numpy as np

import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from tax_calculator import tax
from GrowthCalculator import GrowthCalculator

# Allow plotly to operate within the notebook
init_notebook_mode()

%load_ext autoreload
%autoreload 2

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
# Setup Simulation Conditions
starting_salary = 100000
starting_capital = 10000
number_of_years = 10

In [99]:
# V1: Dividend Stocks
divs = GrowthCalculator(
        salary = starting_salary,
        starting_investment = starting_capital,
        capital_growth = 0.02,
        dividend_payout = 0.07,
        years = number_of_years
    )

divs.calculate_growth(final_year_liquidation=False)

df_divs = pd.DataFrame(divs.investment_history)
# df_divs

In [101]:
# V2: Growth Stocks without liquidation
growth_hold = GrowthCalculator(
        salary = starting_salary,
        starting_investment = starting_capital,
        capital_growth = 0.07,
        dividend_payout = 0.02,
        years = number_of_years
    )

growth_hold.calculate_growth(final_year_liquidation=False)

df_growth_hold = pd.DataFrame(growth_hold.investment_history)
# df_growth_hold

In [105]:
# V3: Growth Stocks with liquidation
growth_sell = GrowthCalculator(
        salary = starting_salary,
        starting_investment = starting_capital,
        capital_growth = 0.07,
        dividend_payout = 0.02,
        years = number_of_years
    )

growth_sell.calculate_growth(final_year_liquidation=True)

df_growth_sell = pd.DataFrame(growth_sell.investment_history)
df_growth_sell

,capital,dividend_income,excess_tax,salary,year
0,10000.000000,0.000000,0.000000,100000,0
1,10914.000000,200.000000,74.000000,100000,1
2,11911.539600,218.280000,80.763600,100000,2
3,13000.254319,238.230792,88.145393,100000,3
4,14188.477564,260.005086,96.201882,100000,4
5,15485.304414,283.769551,104.994734,100000,5
6,16900.661237,309.706088,114.591253,100000,6
7,18445.381674,338.013225,125.064893,100000,7
8,20131.289559,368.907633,136.495824,100000,8
9,21971.289425,402.625791,148.971543,100000,9


In [104]:
print("Dividend Plan")
print(divs.summarised)

print('')

print("Growth Plan - Hold")
print(growth_hold.summarised)

print('')

print("Growth Plan - Sell")
print(growth_sell.summarised)

Dividend Plan
{'Total Tax Paid': 3961.36, 'Total Earnings': 10706.37, 'Total Capital': 23979.47, 'Final Position': 20018.11}

Growth Plan - Hold
{'Total Tax Paid': 1131.82, 'Total Earnings': 3058.96, 'Total Capital': 23979.47, 'Final Position': 22847.65}

Growth Plan - Sell
{'Total Tax Paid': 3105.19, 'Total Earnings': 3058.96, 'Total Capital': 21843.51, 'Final Position': 21843.51}
